In [1]:
import numpy as np
import tensorflow as tf

AUDIO_SEGMENT_LEN_FRAMES = 172
NOTES_TOTAL_BINS = 88

In [2]:
filenames = ["split_000.tfrecord"]

raw_dataset = tf.data.TFRecordDataset(filenames)

In [5]:

for raw_record in raw_dataset.take(1):
    parsed_example = tf.io.parse_single_example(raw_record, {
        "id": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "X_spec": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "X_contours": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "contours_len": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        "X_notes": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "notes_len": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        "X_onsets": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "onsets_len": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    } )

    print(parsed_example['id'])
    print(parsed_example['contours_len'])

    print(tf.ones(parsed_example['contours_len'], dtype=tf.float32))
    print(tf.io.parse_tensor(parsed_example['X_contours'], out_type=tf.int64))



tf.Tensor(b'03_SS1-68-E_comp-seg00', shape=(), dtype=string)
tf.Tensor(489, shape=(), dtype=int64)
tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1

In [10]:
def process_example(example_proto):
    parsed_example = tf.io.parse_single_example(example_proto, {
        "id": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "X_spec": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "X_contours": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "contours_len": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        "X_notes": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "notes_len": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        "X_onsets": tf.io.FixedLenFeature([], tf.string, default_value=''),
        "onsets_len": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    } )

    id = parsed_example['id']
    X_spec = tf.io.parse_tensor(parsed_example['X_spec'], out_type=tf.float32)
    
    contours_coords = tf.io.parse_tensor(parsed_example['X_contours'], out_type=tf.int64)
    notes_coords = tf.io.parse_tensor(parsed_example['X_notes'], out_type=tf.int64)
    onsets_coords = tf.io.parse_tensor(parsed_example['X_onsets'], out_type=tf.int64)
    
    X_contours = tf.sparse.to_dense(tf.sparse.SparseTensor(contours_coords, tf.ones(parsed_example['contours_len'], dtype=tf.float32), [AUDIO_SEGMENT_LEN_FRAMES, 3*NOTES_TOTAL_BINS]))
    X_notes = tf.sparse.to_dense(tf.sparse.SparseTensor(notes_coords, tf.ones(parsed_example['notes_len'], dtype=tf.float32), [AUDIO_SEGMENT_LEN_FRAMES, NOTES_TOTAL_BINS]))
    X_onsets = tf.sparse.to_dense(tf.sparse.SparseTensor(onsets_coords, tf.ones(parsed_example['onsets_len'], dtype=tf.float32), [AUDIO_SEGMENT_LEN_FRAMES, NOTES_TOTAL_BINS]))

    return id, X_spec, X_contours, X_notes, X_onsets

dataset = raw_dataset.map(process_example)

for record in dataset.take(3):
    print(record)

(<tf.Tensor: shape=(), dtype=string, numpy=b'03_SS1-68-E_comp-seg00'>, <tf.Tensor: shape=(172, 309), dtype=float32, numpy=
array([[-51.55394 , -45.05177 , -46.79663 , ..., -64.19376 , -64.19376 ,
        -64.19376 ],
       [-51.490665, -44.862705, -46.473396, ..., -64.19376 , -64.19376 ,
        -64.19376 ],
       [-51.447273, -44.673264, -46.16056 , ..., -64.19376 , -64.19376 ,
        -64.19376 ],
       ...,
       [-39.53833 , -38.624344, -49.48183 , ..., -64.19376 , -64.19376 ,
        -64.19376 ],
       [-39.510616, -38.60681 , -49.34308 , ..., -64.19376 , -64.19376 ,
        -64.19376 ],
       [-39.46465 , -38.606094, -49.027554, ..., -64.19376 , -64.19376 ,
        -64.19376 ]], dtype=float32)>, <tf.Tensor: shape=(172, 264), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 